# Übung 1.2 - Lineare Regression (LASSO) am Beispiel Presshärten 
VO Maschinelles Lernen in der Produktion, WS2020/21, Moritz von Unold, Richard Lux, Felix Soest

#### In diesem Notebook wird das Verfahren Lineare Regression mit LASSO anhand des Anwendungsbeispiels Presshärten geübt.

![alt text](Prozess_Modellentwicklung_v2.png "Title")

#### 0. Bibliotheken importieren

#### 1. Daten erfassen
- Daten importieren

#### 2. Daten erkunden
- Daten tabellarisch darstellen
- statistische Kennzahlen der Daten berechnen

#### 3. Daten vorbereiten
- Daten in Trainings- und Testdaten aufteilen

#### 4. Modelle bilden (hier nur 1 Modell)
- Mögliche Hyperparameter (Modell-Parameter) anzeigen
- Modell erstellen (untrainiert)
- Modell anhand Trainingsdaten trainieren

#### 5. (Modelle validieren)
- Da nur ein Modell erstellt wird entfällt dieser Schritt

#### 6. Modell testen & anwenden
- Berechnung der Modellvorhersage auf den Testdaten
- Bewertung der Modellgüte mittels Graph (Y/Y_pred)
- Bewertung der Modellgüte mittles MAE

#### 7. Visualisierung Zusammenhang alpha-Wert und MAE
- Trainieren und Testen von 30 Modellen mit unterschiedlichen alpha-Werten

### 0. Bibliotheken importieren

In [ ]:
# 0. Code-Block

# Importiere benötigte Bibliotheken
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

pd.options.mode.chained_assignment = None
%matplotlib inline

### 1. Daten erfassen - Daten importieren

TODO:
- Wähle eine Zahl zwischen 1 und 100 für die Generierung deiner spezifischen Zufallszahlen  (1. Code-Block - Zeile 8)
- Ausführen des 1. Code-Blocks (markieren und "Run" drücken)

AUSGABE:
- Gewählte Zufallszahl

In [ ]:
# 1. Code-Block

# Erstelle eigene Zufallszahlen
my_seed = TODO

# Lade Datensatz
df = pd.read_excel("Presshärten_verrauscht_normiert.xlsx")

# Ausgabe gewählte Zufallszahlen
print("\nGewählte Zahl für Zufallszahlen: \t" + str(my_seed))

### 2.1 Daten erkunden - Daten tabellarisch darstellen

TODO:
- Stell ein wie viele der letzten Tabellen-Zeilen du anzeigen möchtest (2. Code-Block - Zeile 4)
- Ausführen des 2. Code-Blocks (markieren und "Run" drücken)

AUSGABE:
- Tabelle des Datensatzes

In [ ]:
# 2. Code-Block

# Ausgabe der Daten in tabellarischer Form
df.tail(TODO)

### 2.2 Daten erkunden - statistische Kennzahlen der Daten berechnen

Ausgabe der statistischen Kennzahlen aller Variablen.

TODO:
- Ausführen des 3. Code-Blocks (markieren und "Run" drücken)

AUSGABE:
- statistische Kennzahlen

In [ ]:
# 3. Code-Block

# Zeige statistische Kennzahlen aller Variablen an
df.describe()

### 3.1 Daten vorbereiten - Aufteilen der Daten in Trainings- und Testdaten

Der Datensatz wird in einen Trainingsdatensatz und einen Testdatensatz aufgeteilt:
Übliche Werte sind für den Trainingsdatensatz 70 - 80% der Daten und für den Testdatensatz 20 - 30%.

Mit dem Trainingsdatensatz wird das Regressionsmodell trainiert, der Testdatensatz dient dazu das trainierte Modell zu testen.

TODO:
- Setzte für train_size einen Wert zwischen 0.1 und 0.9 ein (4. Code-Block - Zeile 4)
- Beispiel: Ein Wert von 0.8 bedeutet der Datensatz wird in 80% Trainingsdaten und 20% Testdaten unterteilt
- Ausführen des 4. Code-Blocks (markieren und "Run" drücken)

AUSGABE:
- Größe der Datensätze
- Ersten 3 Zeilen von X und von y

In [ ]:
# 4. Code-Block

# Verhältnis Trainings- und Testdaten
train_size = TODO

# Teile Datensatz in Trainings- und Testdatensatz auf
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(
    df.drop(columns=["Bauteilhärte"]),
    df["Bauteilhärte"],
    test_size=(1 - train_size),
    random_state=my_seed,
)

# Ausgabe Datensätze und Anzahl Datenpunkte
print(
    "\nAnzahl Traingsdaten: \t"
    + str(len(y_train))
    + " / "
    + str(len(df))
    + " \t("
    + str("%.2f" % (100 * len(y_train) / len(df)))
    + "%)"
)
print(
    "Anzahl Testdaten: \t"
    + str(len(y_test))
    + " / "
    + str(len(df))
    + " \t("
    + str("%.2f" % (100 * len(y_test) / len(df)))
    + "%)"
)
print("\nX: \n" + str(X_train.head(3)))
print("\ny:\tBauteilhärte\n" + str(y_train.head(3)))

### 4.1 Modelle bilden - Mögliche Hyperparameter anzeigen

Vor der Erstellung eines Modells lassen wir uns zunächst alle einstellbaren Hyperparameter anzeigen.

TODO:
- Ausführen des 5. Code-Blocks (markieren und "Run" drücken)

AUSGABE:
- Mögliche Hyperparameter für die Erstellung des Linearen Modells

In [ ]:
# 5. Code-Block

# Ausgabe möglicher Hyperparameter
from sklearn import linear_model

linear_model.Lasso().get_params()

Beschreibung der Hyperparameter:
http://scikit-learn.org/stable/modules/generated/sklearn.linear_model.Lasso.html

### 4.2 Modelle bilden - Modell erstellen (untrainiert)

Wir erstellen ein Lineares LASSO-Modell. Diese enthält den zusätzlichen Hyperparameter 'alpha'. Alpha ist der Strafterm welcher das ausklammern von Variablen mit geringem Einfluss auf das Ergebnis ermöglicht.

Als Hyperparameter werden neben dem Strafterm alpha, 'fit_intercept' als True eingestellt, da wir ein Regressionsmodell MIT Konstante berechnen wollen. Des Weiteren wird 'random_state' auf 'my_seed' eingestellt damit der Algorithmus mit unseren spezifischen Zufallszahlen arbeitet.

TODO:
- Wähle einen alpha-Wert zwischen 2 und 0.0000001, z.B: 0.1 (6. Code-Block - Zeile 4) 
- Setzte in die Klammer "fit_intercept=True" ein (6. Code-Block - Zeile 8)
- Setzte in die Klammer "random_state=my_seed" ein (6. Code-Block - Zeile 8)
- Setzte in die Klammer "alpha=alpha" ein (6. Code-Block - Zeile 8)
- Ausführen des 6. Code-Blocks (markieren und "Run" drücken)

AUSGABE:
- Hyperparameter des erstellten Modells

Versuche den alpha-Wert so einzustellen das das Modell aus 2-4 Variablen besteht (Anzahl Variablen im Modell siehe Ausgabe 4.3).

In [ ]:
# 6. Code-Block

# Einstellen des alpha-Parameters (= Lasso-Strafterm)
alpha = TODO

# Erstelle Lineares LASSO-Modell
lasso = linear_model.Lasso(TODO, TODO, TODO)

# Ausgabe Hyperparameter des Modells
lasso.get_params()

### 4.3 Modelle bilden - Modelle anhand Trainingsdaten trainieren

Während des Trainings werden die Parameter des Modells/der Regressionsgerade angepasst sodass dieses die Daten des Trainingsdatensatzes abbildet (Gradientenverfahren). 

TODO:
- Ausführen des 7. Code-Blocks (markieren und "Run" drücken)

AUSGABE:
- Gleichung der Regressionsgeraden
- Graph: Einfluss der Input-Variablen auf die Bauteilhärte

In [ ]:
# 7. Code-Block

# Trainieren des Modells
lasso.fit(X_train, y_train)

# Ausgabe des Modells
model_1 = "y = " + str("%.3f" % lasso.intercept_)
for i in range(len(lasso.coef_)):
    model_1 += (
        " + "
        + str("%.3f" % lasso.coef_[i])
        + " * "
        + str(list(X_train)[i])
    )
print("\nRegressionsmodell: \n" + str(model_1))

# Ausgabe Graph
fig = plt.figure(figsize=(16, 6))
plt.bar(list(X_train), lasso.coef_, align="center")
plt.title("Einfluss der Variablen auf die Bauteilhärte")

Wie viele Variablen sind noch im Modell enthalten (wurden also nicht durch den Lasso-Strafterm 0 gesetzt)?

Versuche durch die Wahl eines geeigneten Lasso-Wertes (alpha) in Punkt 4.2 ein Modell mit 2-4 Variablen zu erstellen.

### 5. Validieren der Modelle - entfällt

Dieser Schritt ist nur nötig wenn mehrere Modelle erstellt werden.

### 6.1 Modell testen & anwenden - Berechnung der Modellvorhersage auf den Testdaten

Mit dem erstellten Modell wird eine Vorhersage für alle Testdaten berechnet und diese mit den realen Werten verglichen.

TODO:
- Stelle ein wie viele Zeilen des Ergebnisses du anzeigen lassen willst (8. Code-Block - Zeile 12)
- Ausführen des 8. Code-Blocks (markieren und "Run" drücken)

AUSGABE:
- Tabelle Reale Bauteilhärte, Modell-Vorhersage der Bauteilhärte sowie die Abweichung

In [ ]:
# 8. Code-Block

# Berechne Modellvorhersage für jede Zeile des Test-Datensatzes
results_test = pd.DataFrame(y_test)
results_test["Vorhersage_Bauteilhärte"] = lasso.predict(X_test)

# Berechne Differenz: Realer Wert - Vorhersage
results_test["Abweichung_Vorhersage"] = (
    results_test["Bauteilhärte"] - results_test["Vorhersage_Bauteilhärte"]
)

# Ausgabe Tabelle
print("\nModell-Vorhersage und reale Werte der Testdaten:")
results_test.head(TODO)

### 6.2 Modell testen & anwenden - Bewertung der Modellgüte mittels Graph (Y/Y_pred)

Graph der Modell-Vorhersage über den realen Werten.

TODO:
- Stelle die Größe des auszugebenden Graphen ein, z.B. figsize=(9, 8) (9. Code-Block - Zeile 5)

(Die Größe ist in Zoll -> 1 Zoll = 2,54 cm)
- Ausführen des 9. Code-Blocks (markieren und "Run" drücken)

AUSGABE:
- Tabelle mit realer Bauteilhärte, Modell-Vorhersage der Bauteilhärte sowie die Abweichung

In [ ]:
# 9. Code-Block

# Ausgabe Graph
min_test, max_test = y_test.min(), y_test.max()
fig = plt.figure(figsize=(TODO, TODO))
plt.plot(y_test, np.squeeze(lasso.predict(X_test)), "o", alpha=0.4)
plt.plot([min_test, max_test], [min_test, max_test], "--", c=(0, 0, 0))
plt.xlim(min_test - 0.2 * (max_test - min_test), max_test + 0.2 * (max_test - min_test))
plt.ylim(min_test - 0.2 * (max_test - min_test), max_test + 0.2 * (max_test - min_test))
plt.xlabel("reale Bauteilhärte")
plt.ylabel("vorhergesagte Bauteilhärte")
plt.title("Lineare Regression Lasso - Testdatensatz")

### 6.3 Modell testen & anwenden - Berechnung MAE

Um die Güte des Regressionsmodells zu bestimmen wird der MAE (Mean-Absolut-Error) berechnet. 

TODO:
- Ausführen des 10. Code-Blocks (markieren und "Run" drücken)

AUSGABE:
- MAE

In [ ]:
# 10. Code-Block

# Berechne MAE
from sklearn.metrics import mean_absolute_error

y_pred = lasso.predict(X_test)
mae = mean_absolute_error(y_test, y_pred)

# Ausgabe
print("\nMAE: \t" + str("%.5f" % mae))

### 7. Visualisierung Zusammenhang Alpha-Wert und MAE

Um den Einfluss des alpha-Wertes zu beurteilen werden 80 Modelle mit verschiedenen alpha-Werten trainiert und jeweils der RMSE berechnet.

TODO:
- Wähle den niedrigsten und höchsten alpha-Wert (alpha_min, alpha_max), es werden 80 Modelle mit varrierendem alpha-Wert beginnend beim niedrigsten und endend beim höchsten alpha-Wert berechnet (10. Code-Block - Zeile 4 & 5)
- Beginne mit einer großen Spanne für alpha (z.B. 0.0001 - 3) und taste dich schrittweise an das Optimum an.
- Ausführen des 11. Code-Blocks (markieren und "Run" drücken)

AUSGABE:
- Graph: Zusammenhang Alpha-Wert und RMSE (bzw. Modellgüte)

In [ ]:
# 11. Code-Block

# Einstellen des Lasso-Strafterms/alpha-Wertes
alpha_min = TODO  # Nicht 0 einsetzten, z.B. 0.0000001
alpha_max = TODO  # z.B. 2.0

# Trainiere 80 Modelle mit Hyperparameter alpha zwischen lasso_begin und lasso_end
list_MAE = []
list_varcount = []
alphas = np.linspace(alpha_min, alpha_max, 80)
for alpha in alphas:

    # Erstelle und Trainiere Lineares Modell
    lasso = linear_model.Lasso(alpha=(alpha))
    lasso.fit(X_train, y_train)

    # Berechne Vorhersage und RMSE
    y_P_test = lasso.predict(X_test)
    y_pred = lasso.predict(X_test)
    list_MAE.append(mean_absolute_error(y_test, y_pred))

    # Zähle Variablen nicht 0
    coefs = 0
    for coef in lasso.coef_:
        if coef != 0:
            coefs += 1
    list_varcount.append(coefs)

# Ausgabe Graphen
idx_min = list_MAE.index(min(list_MAE))
plt.figure(figsize=(16, 13))
plt.subplot(2, 1, 1)
plt.plot(alphas, list_MAE, "o--")
plt.xlabel("Alpha-Werte der Modelle ")
plt.ylabel("MAE")
plt.title("Modellgüte (MAE) in Abhängigkeit des alpha-Wertes (Lasso-Strafterm)")
plt.plot(alphas[idx_min], list_MAE[idx_min], "ro")
plt.subplot(2, 1, 2)
plt.plot(alphas, list_varcount, "o--")
plt.xlabel("Alpha-Werte der Modelle")
plt.ylabel("Anzahl Varibalen im Modell")
plt.title(
    "Anzahl Variablen im Modell in Abhängigkeit des alpha-Wertes (Lasso-Strafterm)"
)
plt.plot(alphas[idx_min], list_varcount[idx_min], "ro")

# Ausgabe bester alpha-Wert
print(
    "\nGeringster MAE (höchste Genauigkeit des Modells) bei einem alpha-Wert von "
    + str("%.5f" % alphas[idx_min])
    + "."
)

### Verständnisfragen:

1. Welche Vorteile/Unterschiede besitzt die Lineare Regression mit LASSO gegenüber der Linearen Regression?
2. Welcher Alpha-Wert ergibt den geringsten MAE für das Problem in diesem Notebook? Wie viele und welche Variablen enthält das Modell mit diesem alpha-Wert? Welchen MAE hat das Modell?